In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import random as rn
keras = tf.keras

In [2]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
path_winn = r'C:\Users\adria\1Python\Meu_novo_projeto\WIN_N_1K\WIN_N_1k.csv'
winn = pd.read_csv(path_winn, index_col=0, parse_dates=True)

In [5]:
path_wing = r'C:\Users\adria\1Python\Meu_novo_projeto\WING23\WING23.csv'
wing = pd.read_csv(path_wing, index_col=0, parse_dates=True)

In [6]:
path_winj = r'C:\Users\adria\1Python\Meu_novo_projeto\WINJ23\WINJ23.csv'
winj = pd.read_csv(path_winj, index_col=0, parse_dates=True)

# Trasforma os dos dados

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
lag = 2

### Treino

In [8]:
winn['hl'] = (winn['high'] + winn['low'] + winn['close'] + winn['close']) / 4
winn['mvhl'] = winn['hl'].rolling(lag).mean()

In [9]:
winn = winn.drop(['open', 'tick_volume','real_volume'], axis=1)

In [10]:
train = winn.dropna()

In [11]:
train = scaler.fit_transform(train)

### Validacao

In [12]:
wing['hl'] = (wing['high'] + wing['low'] + wing['close'] + wing['close']) / 4
wing['mvhl'] = wing['hl'].rolling(lag).mean()

In [13]:
wing = wing.drop(['open', 'tick_volume','real_volume'], axis=1)

In [14]:
valid = wing.dropna()

In [15]:
valid = scaler.transform(valid)

### Teste

In [16]:
winj['hl'] = (winj['high'] + winj['low'] + winj['close'] + winj['close']) / 4
winj['mvhl'] = winj['hl'].rolling(lag).mean()
winj = winj.drop(['open', 'tick_volume','real_volume'], axis=1)
test = winj.dropna()
test = scaler.transform(test)

# Preprocessamento

In [17]:
batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1

In [18]:
def sliding_window_sequence(data, window_size, num_inputs, num_targets, stride, batch_size, sampling_rate=1):

    num_examples = (len(data) - window_size) // stride
    num_batches = num_examples // batch_size
    inputs = np.zeros((num_examples, window_size, num_inputs))
    targets = np.zeros((num_examples, num_targets))
    
    for i in range(num_examples):
        idx = i * stride
        inputs[i] = data[idx : idx+window_size : sampling_rate, :num_inputs]
        targets[i] = data[idx+window_size : idx+window_size+num_targets, -1]
    
    batch_inputs = np.zeros((batch_size, window_size, num_inputs))
    batch_targets = np.zeros((batch_size, num_targets))
    while True:
        for b in range(num_batches):
            batch_start = b * batch_size
            batch_inputs = inputs[batch_start:batch_start+batch_size]
            batch_targets = targets[batch_start:batch_start+batch_size]
            yield batch_inputs, batch_targets

In [ ]:
 def sliding_window_sequence(data, window_size, num_inputs, num_targets, stride, batch_size, sampling_rate=1):
    # Calcula o número total de exemplos que podem ser gerados usando a janela de tamanho fixo
    num_examples = (len(data) - window_size) // stride
    
    # Calcula o número de lotes (batches) que podem ser gerados
    num_batches = num_examples // batch_size
    
    # Cria matrizes numpy para armazenar inputs e targets
    inputs = np.zeros((num_examples, window_size, num_inputs))
    targets = np.zeros((num_examples, num_targets))
    
    # Loop através de cada exemplo e armazena-os nas matrizes inputs e targets
    for i in range(num_examples):
        idx = i * stride  # Calcula o índice inicial do exemplo atual
        # Extrai a sequência de entrada do exemplo atual e armazena em inputs
        inputs[i] = data[idx : idx+window_size : sampling_rate, :num_inputs]
        # Extrai a sequência de targets do exemplo atual e armazena em targets
        targets[i] = data[idx+window_size : idx+window_size+num_targets, -1]
        #targets[i] = data[idx+window_size : idx+window_size+num_targets, :num_inputs] # todos os valores das linhas
    
    # Cria matrizes numpy para armazenar os inputs e targets de cada batch
    batch_inputs = np.zeros((batch_size, window_size, num_inputs))
    batch_targets = np.zeros((batch_size, num_targets))
    
    # Loop infinito para gerar batches de dados
    while True:
        # Loop através de cada batch
        for b in range(num_batches):
            batch_start = b * batch_size  # Calcula o índice inicial do batch atual
            # Extrai os inputs do batch atual a partir da matriz inputs
            batch_inputs = inputs[batch_start:batch_start+batch_size]
            # Extrai os targets do batch atual a partir da matriz targets
            batch_targets = targets[batch_start:batch_start+batch_size]
            # Gera um batch de dados contendo inputs e targets
            yield batch_inputs, batch_targets


In [19]:
train_set = sliding_window_sequence(train, window_size=window_size, num_inputs=num_inputs, num_targets=num_targets, stride=stride, batch_size=batch_size, sampling_rate=sampling_rate)
valid_set = sliding_window_sequence(valid, window_size=window_size, num_inputs=num_inputs, num_targets=num_targets, stride=stride, batch_size=batch_size, sampling_rate=sampling_rate)
test_set = sliding_window_sequence(test, window_size=window_size, num_inputs=num_inputs, num_targets=num_targets, stride=stride, batch_size=batch_size, sampling_rate=sampling_rate)

# Treino

In [20]:
num_inputs

5

In [48]:
%%time


#________________________________ LSTM

name_model = "s2s_LSTM_256TimeD"

keras.backend.clear_session()

# Definir hiperparâmetros
learning_rate = 0.0001
l2_reg = 0.00001
dropout_rate = 0.001
func_act='sigmoid'

batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1



# Definir a entrada da rede
inputs = keras.layers.Input(shape=[None, num_inputs])

# LSTM
reshaped_inputs = keras.layers.Reshape((-1, num_inputs))(inputs)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)

# TimeDistributed Dense
dense = keras.layers.TimeDistributed(keras.layers.Dense(64))(lstm)
dense = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(dense)

# LSTM
lstm = keras.layers.LSTM(256, return_sequences=False, kernel_regularizer=keras.regularizers.L2(l2_reg))(dense)
output_lstm = keras.layers.Dense(num_targets)(lstm)

# Definir o modelo com a entrada e saída
model = keras.models.Model(inputs=inputs, outputs=output_lstm)

# Definir o otimizador com taxa de aprendizado definida
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


# Compilar o modelo com a função de perda, otimizador e métrica desejados
model.compile(loss=keras.losses.logcosh, optimizer=optimizer, metrics=["mse"])

# Adicionar callbacks para o modelo
model_checkpoint = keras.callbacks.ModelCheckpoint(name_model, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=32)

# Treinar o modelo
model.fit(train_set,
          steps_per_epoch= len(train)//batch_size,
          epochs=500,
          validation_data=valid_set,
          validation_steps=len(valid)//batch_size,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/500
781/781 [==============================] - 24s 26ms/step - loss: 0.0031 - mse: 0.0041 - val_loss: 0.0015 - val_mse: 0.0012


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD\assets


Epoch 2/500
781/781 [==============================] - 16s 20ms/step - loss: 0.0011 - mse: 5.5543e-04 - val_loss: 0.0014 - val_mse: 0.0014


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD\assets


...

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD\assets


Epoch 122/500
781/781 [==============================] - 16s 21ms/step - loss: 2.9042e-05 - mse: 3.3635e-05 - val_loss: 3.0068e-05 - val_mse: 3.5794e-05
Epoch 123/500
781/781 [==============================] - 16s 20ms/step - loss: 4.0362e-05 - mse: 5.6502e-05 - val_loss: 3.3553e-05 - val_mse: 4.2831e-05
Epoch 124/500
781/781 [==============================] - 16s 21ms/step - loss: 3.1820e-05 - mse: 3.9453e-05 - val_loss: 2.4083e-05 - val_mse: 2.4040e-05


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD\assets


Epoch 125/500
781/781 [==============================] - 17s 21ms/step - loss: 3.0750e-05 - mse: 3.7562e-05 - val_loss: 3.2896e-05 - val_mse: 4.1960e-05
Epoch 126/500
781/781 [==============================] - 16s 20ms/step - loss: 3.9995e-05 - mse: 5.6209e-05 - val_loss: 3.1135e-05 - val_mse: 3.8477e-05
Epoch 127/500
781/781 [==============================] - 16s 20ms/step - loss: 3.0334e-05 - mse: 3.6939e-05 - val_loss: 2.6715e-05 - val_mse: 2.9799e-05
Epoch 128/500
781/781 [==============================] - 16s 20ms/step - loss: 3.2054e-05 - mse: 4.0607e-05 - val_loss: 5.2155e-05 - val_mse: 8.0954e-05
Epoch 129/500
781/781 [==============================] - 16s 21ms/step - loss: 3.8917e-05 - mse: 5.4432e-05 - val_loss: 4.1248e-05 - val_mse: 5.9151e-05
Epoch 130/500
781/781 [==============================] - 17s 21ms/step - loss: 2.8079e-05 - mse: 3.2837e-05 - val_loss: 4.6202e-05 - val_mse: 6.9306e-05
Epoch 131/500
781/781 [==============================] - 17s 21ms/step - loss: 3.4

# Resultados

In [49]:
name_model

's2s_LSTM_256TimeD'

In [50]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 2s 11ms/step - loss: 1.0467e-04 - mse: 1.8949e-04


In [51]:
model = keras.models.load_model(name_model)

In [52]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 3s 11ms/step - loss: 4.4455e-05 - mse: 6.4795e-05


In [55]:
%%time


#________________________________ LSTM

name_model = "s2s_LSTM_256TimeD_2"

keras.backend.clear_session()

# Definir hiperparâmetros
learning_rate = 0.0001
l2_reg = 0.00001
dropout_rate = 0.001
func_act='sigmoid'

batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1



# Definir a entrada da rede
inputs = keras.layers.Input(shape=[None, num_inputs])

# LSTM
reshaped_inputs = keras.layers.Reshape((-1, num_inputs))(inputs)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)

# TimeDistributed Dense
dense = keras.layers.TimeDistributed(keras.layers.Dense(256))(lstm)
dense = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(dense)

# LSTM
lstm = keras.layers.LSTM(256, return_sequences=False, kernel_regularizer=keras.regularizers.L2(l2_reg))(dense)
output_lstm = keras.layers.Dense(num_targets)(lstm)

# Definir o modelo com a entrada e saída
model = keras.models.Model(inputs=inputs, outputs=output_lstm)

# Definir o otimizador com taxa de aprendizado definida
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


# Compilar o modelo com a função de perda, otimizador e métrica desejados
model.compile(loss=keras.losses.logcosh, optimizer=optimizer, metrics=["mse"])

# Adicionar callbacks para o modelo
model_checkpoint = keras.callbacks.ModelCheckpoint(name_model, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=32)

# Treinar o modelo
model.fit(train_set,
          steps_per_epoch= len(train)//batch_size,
          epochs=500,
          validation_data=valid_set,
          validation_steps=len(valid)//batch_size,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/500
781/781 [==============================] - 22s 23ms/step - loss: 0.0029 - mse: 5.8464e-04 - val_loss: 0.0060 - val_mse: 0.0086


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_2\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_2\assets


Epoch 2/500
781/781 [==============================] - 18s 23ms/step - loss: 0.0017 - mse: 6.9239e-04 - val_loss: 0.0056 - val_mse: 0.0088


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_2\assets


...

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_2\assets


Epoch 81/500
781/781 [==============================] - 17s 21ms/step - loss: 4.3999e-05 - mse: 5.3076e-05 - val_loss: 3.5753e-05 - val_mse: 3.6743e-05
Epoch 82/500
781/781 [==============================] - 16s 21ms/step - loss: 5.0359e-05 - mse: 6.6263e-05 - val_loss: 6.6174e-05 - val_mse: 9.7766e-05
Epoch 83/500
781/781 [==============================] - 16s 21ms/step - loss: 5.1298e-05 - mse: 6.8330e-05 - val_loss: 9.8917e-05 - val_mse: 1.6325e-04
Epoch 84/500
781/781 [==============================] - 16s 21ms/step - loss: 4.1968e-05 - mse: 4.9810e-05 - val_loss: 1.3356e-04 - val_mse: 2.3268e-04
Epoch 85/500
781/781 [==============================] - 16s 21ms/step - loss: 4.5781e-05 - mse: 5.7946e-05 - val_loss: 2.4323e-04 - val_mse: 4.5230e-04
Epoch 86/500
781/781 [==============================] - 16s 21ms/step - loss: 5.0032e-05 - mse: 6.6718e-05 - val_loss: 2.4886e-04 - val_mse: 4.6379e-04
Epoch 87/500
781/781 [==============================] - 16s 21ms/step - loss: 4.4106e-05

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_2\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_2\assets


Epoch 92/500
781/781 [==============================] - 17s 21ms/step - loss: 4.3011e-05 - mse: 5.4285e-05 - val_loss: 4.2108e-05 - val_mse: 5.2629e-05
Epoch 93/500
781/781 [==============================] - 18s 22ms/step - loss: 4.7474e-05 - mse: 6.3463e-05 - val_loss: 5.4642e-05 - val_mse: 7.7945e-05
Epoch 94/500
781/781 [==============================] - 18s 23ms/step - loss: 4.2751e-05 - mse: 5.4109e-05 - val_loss: 6.4754e-05 - val_mse: 9.8347e-05
Epoch 95/500
781/781 [==============================] - 18s 23ms/step - loss: 3.8991e-05 - mse: 4.6852e-05 - val_loss: 1.0763e-04 - val_mse: 1.8458e-04
Epoch 96/500
781/781 [==============================] - 18s 23ms/step - loss: 4.4069e-05 - mse: 5.7396e-05 - val_loss: 1.4570e-04 - val_mse: 2.6106e-04
Epoch 97/500
781/781 [==============================] - 18s 23ms/step - loss: 4.5893e-05 - mse: 6.1202e-05 - val_loss: 1.3253e-04 - val_mse: 2.3484e-04
Epoch 98/500
781/781 [==============================] - 18s 23ms/step - loss: 3.9255e-05

In [56]:
name_model

's2s_LSTM_256TimeD_2'

In [57]:
model = keras.models.load_model(name_model)

In [58]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 3s 11ms/step - loss: 3.6565e-05 - mse: 4.1042e-05


In [60]:
%%time


#________________________________ LSTM

name_model = "s2s_LSTM_256TimeD_3"

keras.backend.clear_session()

# Definir hiperparâmetros
learning_rate = 0.0001
l2_reg = 0.00001
dropout_rate = 0.001
func_act='sigmoid'

batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1



# Definir a entrada da rede
inputs = keras.layers.Input(shape=[None, num_inputs])

# LSTM
reshaped_inputs = keras.layers.Reshape((-1, num_inputs))(inputs)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.GRU(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)

# TimeDistributed Dense
timed = keras.layers.TimeDistributed(keras.layers.Dense(256))(lstm)
timed = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(timed)
timed = keras.layers.TimeDistributed(keras.layers.Dense(256))(lstm)
timed = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(timed)

# LSTM
lstm = keras.layers.LSTM(256, return_sequences=False, kernel_regularizer=keras.regularizers.L2(l2_reg))(timed)
output_lstm = keras.layers.Dense(num_targets)(lstm)


# Definir o modelo com a entrada e saída
model = keras.models.Model(inputs=inputs, outputs=output_lstm)

# Definir o otimizador com taxa de aprendizado definida
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


# Compilar o modelo com a função de perda, otimizador e métrica desejados
model.compile(loss=keras.losses.logcosh, optimizer=optimizer, metrics=["mse"])

# Adicionar callbacks para o modelo
model_checkpoint = keras.callbacks.ModelCheckpoint(name_model, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=32)

# Treinar o modelo
model.fit(train_set,
          steps_per_epoch= len(train)//batch_size,
          epochs=500,
          validation_data=valid_set,
          validation_steps=len(valid)//batch_size,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/500
781/781 [==============================] - 20s 21ms/step - loss: 0.0049 - mse: 0.0033 - val_loss: 0.0026 - val_mse: 2.3855e-04


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_3\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_3\assets


Epoch 2/500
781/781 [==============================] - 17s 22ms/step - loss: 0.0021 - mse: 1.6858e-04 - val_loss: 0.0017 - val_mse: 2.8678e-04


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_3\assets


...

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_3\assets


Epoch 89/500
781/781 [==============================] - 17s 21ms/step - loss: 4.3916e-05 - mse: 5.4257e-05 - val_loss: 3.3582e-05 - val_mse: 3.3703e-05
Epoch 90/500
781/781 [==============================] - 17s 21ms/step - loss: 4.6212e-05 - mse: 5.9178e-05 - val_loss: 3.6009e-05 - val_mse: 3.8828e-05
Epoch 91/500
781/781 [==============================] - 16s 21ms/step - loss: 4.5090e-05 - mse: 5.7172e-05 - val_loss: 3.4238e-05 - val_mse: 3.5497e-05
Epoch 92/500
781/781 [==============================] - 16s 21ms/step - loss: 4.4109e-05 - mse: 5.5486e-05 - val_loss: 3.7181e-05 - val_mse: 4.1676e-05
Epoch 93/500
781/781 [==============================] - 17s 21ms/step - loss: 4.4806e-05 - mse: 5.7145e-05 - val_loss: 3.8471e-05 - val_mse: 4.4500e-05
Epoch 94/500
781/781 [==============================] - 17s 22ms/step - loss: 4.3444e-05 - mse: 5.4666e-05 - val_loss: 3.6635e-05 - val_mse: 4.1073e-05
Epoch 95/500
781/781 [==============================] - 17s 22ms/step - loss: 4.3546e-05

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_3\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_3\assets


Epoch 102/500
781/781 [==============================] - 18s 22ms/step - loss: 4.1283e-05 - mse: 5.2287e-05 - val_loss: 3.1419e-05 - val_mse: 3.2677e-05
Epoch 103/500
781/781 [==============================] - 17s 22ms/step - loss: 4.2459e-05 - mse: 5.4862e-05 - val_loss: 3.1394e-05 - val_mse: 3.2821e-05
Epoch 104/500
781/781 [==============================] - 17s 22ms/step - loss: 4.0340e-05 - mse: 5.0804e-05 - val_loss: 3.1610e-05 - val_mse: 3.3495e-05
Epoch 105/500
781/781 [==============================] - 17s 22ms/step - loss: 4.0516e-05 - mse: 5.1398e-05 - val_loss: 3.5573e-05 - val_mse: 4.1671e-05
Epoch 106/500
781/781 [==============================] - 18s 23ms/step - loss: 4.0747e-05 - mse: 5.2067e-05 - val_loss: 4.1038e-05 - val_mse: 5.2857e-05
Epoch 107/500
781/781 [==============================] - 17s 22ms/step - loss: 4.1081e-05 - mse: 5.2953e-05 - val_loss: 5.1055e-05 - val_mse: 7.3126e-05
Epoch 108/500
781/781 [==============================] - 17s 22ms/step - loss: 4.0

In [61]:
name_model

's2s_LSTM_256TimeD_3'

In [62]:
model = keras.models.load_model(name_model)

In [63]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 2s 11ms/step - loss: 6.0660e-05 - mse: 9.0866e-05


In [65]:
%%time


#________________________________ LSTM

name_model = "s2s_LSTM_256TimeD_4"

keras.backend.clear_session()

# Definir hiperparâmetros
learning_rate = 0.0001
l2_reg = 0.00001
dropout_rate = 0.001
func_act='sigmoid'

batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1



# Definir a entrada da rede
inputs = keras.layers.Input(shape=[None, num_inputs])

# LSTM
reshaped_inputs = keras.layers.Reshape((-1, num_inputs))(inputs)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)

# TimeDistributed Dense
timed = keras.layers.TimeDistributed(keras.layers.Dense(256))(lstm)
timed = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(timed)


# LSTM
lstm = keras.layers.LSTM(256, return_sequences=False, kernel_regularizer=keras.regularizers.L2(l2_reg))(timed)
output_lstm = keras.layers.Dense(num_targets)(lstm)


# Definir o modelo com a entrada e saída
model = keras.models.Model(inputs=inputs, outputs=output_lstm)

# Definir o otimizador com taxa de aprendizado definida
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


# Compilar o modelo com a função de perda, otimizador e métrica desejados
model.compile(loss=keras.losses.logcosh, optimizer=optimizer, metrics=["mse"])

# Adicionar callbacks para o modelo
model_checkpoint = keras.callbacks.ModelCheckpoint(name_model, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=32)

# Treinar o modelo
model.fit(train_set,
          steps_per_epoch= len(train)//batch_size,
          epochs=500,
          validation_data=valid_set,
          validation_steps=len(valid)//batch_size,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/500
781/781 [==============================] - 21s 23ms/step - loss: 0.0031 - mse: 8.8094e-04 - val_loss: 0.0095 - val_mse: 0.0155


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_4\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_4\assets


Epoch 2/500
781/781 [==============================] - 19s 24ms/step - loss: 0.0020 - mse: 0.0011 - val_loss: 0.0085 - val_mse: 0.0145


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_4\assets


...

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_4\assets


Epoch 212/500
781/781 [==============================] - 18s 23ms/step - loss: 2.9791e-05 - mse: 4.4362e-05 - val_loss: 2.5501e-05 - val_mse: 3.5821e-05
Epoch 213/500
781/781 [==============================] - 18s 24ms/step - loss: 2.8070e-05 - mse: 4.0928e-05 - val_loss: 1.9336e-05 - val_mse: 2.3418e-05
Epoch 214/500
781/781 [==============================] - 19s 24ms/step - loss: 2.5924e-05 - mse: 3.6733e-05 - val_loss: 2.7062e-05 - val_mse: 3.8993e-05
Epoch 215/500
781/781 [==============================] - 18s 24ms/step - loss: 3.1153e-05 - mse: 4.7207e-05 - val_loss: 2.4285e-05 - val_mse: 3.3403e-05
Epoch 216/500
781/781 [==============================] - 19s 24ms/step - loss: 2.4175e-05 - mse: 3.3313e-05 - val_loss: 2.7165e-05 - val_mse: 3.9248e-05
Epoch 217/500
781/781 [==============================] - 18s 23ms/step - loss: 3.0983e-05 - mse: 4.7000e-05 - val_loss: 3.2367e-05 - val_mse: 4.9696e-05
Epoch 218/500
781/781 [==============================] - 19s 24ms/step - loss: 2.5

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_4\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_4\assets


Epoch 226/500
781/781 [==============================] - 18s 23ms/step - loss: 2.4980e-05 - mse: 3.5394e-05 - val_loss: 2.4004e-05 - val_mse: 3.3477e-05
Epoch 227/500
781/781 [==============================] - 19s 24ms/step - loss: 3.0165e-05 - mse: 4.5761e-05 - val_loss: 1.9996e-05 - val_mse: 2.5427e-05
Epoch 228/500
781/781 [==============================] - 18s 23ms/step - loss: 2.3228e-05 - mse: 3.1947e-05 - val_loss: 2.1850e-05 - val_mse: 2.9259e-05
Epoch 229/500
781/781 [==============================] - 18s 23ms/step - loss: 3.0240e-05 - mse: 4.6027e-05 - val_loss: 2.8563e-05 - val_mse: 4.2740e-05
Epoch 230/500
781/781 [==============================] - 18s 24ms/step - loss: 2.3978e-05 - mse: 3.3522e-05 - val_loss: 2.2819e-05 - val_mse: 3.1289e-05
Epoch 231/500
781/781 [==============================] - 18s 23ms/step - loss: 2.7268e-05 - mse: 4.0206e-05 - val_loss: 5.3260e-05 - val_mse: 9.2289e-05
Epoch 232/500
781/781 [==============================] - 18s 23ms/step - loss: 2.7

In [66]:
name_model

's2s_LSTM_256TimeD_4'

In [67]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 2s 11ms/step - loss: 5.9971e-05 - mse: 1.0705e-04


In [68]:
model = keras.models.load_model(name_model)

In [69]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 3s 11ms/step - loss: 2.7936e-05 - mse: 4.1222e-05


In [50]:
%%time


#________________________________ LSTM

name_model = "s2s_LSTM_256TimeD_5"

keras.backend.clear_session()

# Definir hiperparâmetros
learning_rate = 0.0001
l2_reg = 0.00001
dropout_rate = 0.001
func_act='sigmoid'

batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1



# Definir a entrada da rede
inputs = keras.layers.Input(shape=[None, num_inputs])

# LSTM
reshaped_inputs = keras.layers.Reshape((-1, num_inputs))(inputs)
lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
#lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
#lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
#lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)

# TimeDistributed Dense
timed = keras.layers.TimeDistributed(keras.layers.Dense(80))(lstm)
timed = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(timed)


# LSTM
lstm = keras.layers.LSTM(32, return_sequences=False, kernel_regularizer=keras.regularizers.L2(l2_reg))(timed)
output_lstm = keras.layers.Dense(num_targets)(lstm)


# Definir o modelo com a entrada e saída
model = keras.models.Model(inputs=inputs, outputs=output_lstm)

# Definir o otimizador com taxa de aprendizado definida
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


# Compilar o modelo com a função de perda, otimizador e métrica desejados
model.compile(loss=keras.losses.logcosh, optimizer=optimizer, metrics=["mse"])

# Adicionar callbacks para o modelo
model_checkpoint = keras.callbacks.ModelCheckpoint(name_model, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=32)

# Treinar o modelo
model.fit(train_set,
          steps_per_epoch= len(train)//batch_size,
          epochs=500,
          validation_data=valid_set,
          validation_steps=len(valid)//batch_size,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/500
781/781 [==============================] - 16s 17ms/step - loss: 0.0033 - mse: 0.0045 - val_loss: 0.0013 - val_mse: 5.5616e-04


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_5\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_5\assets


Epoch 2/500
781/781 [==============================] - 12s 15ms/step - loss: 0.0012 - mse: 5.3075e-04 - val_loss: 9.6147e-04 - val_mse: 1.5550e-04


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_5\assets


...

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_5\assets


Epoch 149/500
781/781 [==============================] - 15s 20ms/step - loss: 2.9914e-05 - mse: 3.9834e-05 - val_loss: 2.7381e-05 - val_mse: 3.4802e-05
Epoch 150/500
781/781 [==============================] - 16s 20ms/step - loss: 3.1822e-05 - mse: 4.3780e-05 - val_loss: 2.3654e-05 - val_mse: 2.7458e-05


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_5\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_5\assets


Epoch 151/500
781/781 [==============================] - 16s 20ms/step - loss: 2.8586e-05 - mse: 3.7426e-05 - val_loss: 2.4676e-05 - val_mse: 2.9651e-05
Epoch 152/500
781/781 [==============================] - 16s 20ms/step - loss: 3.1691e-05 - mse: 4.3764e-05 - val_loss: 2.4976e-05 - val_mse: 3.0368e-05
Epoch 153/500
781/781 [==============================] - 16s 20ms/step - loss: 2.8673e-05 - mse: 3.7848e-05 - val_loss: 2.4117e-05 - val_mse: 2.8797e-05
Epoch 154/500
781/781 [==============================] - 16s 20ms/step - loss: 3.1074e-05 - mse: 4.2789e-05 - val_loss: 2.7360e-05 - val_mse: 3.5419e-05
Epoch 155/500
781/781 [==============================] - 16s 20ms/step - loss: 2.9054e-05 - mse: 3.8858e-05 - val_loss: 2.5398e-05 - val_mse: 3.1618e-05
Epoch 156/500
781/781 [==============================] - 16s 20ms/step - loss: 2.9524e-05 - mse: 3.9936e-05 - val_loss: 3.3619e-05 - val_mse: 4.8215e-05
Epoch 157/500
781/781 [==============================] - 16s 20ms/step - loss: 2.9

In [51]:
name_model

's2s_LSTM_256TimeD_5'

In [53]:
model = keras.models.load_model(name_model)

In [52]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 2s 10ms/step - loss: 5.6501e-05 - mse: 9.6756e-05


In [54]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 2s 9ms/step - loss: 5.2075e-05 - mse: 8.4311e-05


In [55]:
%%time


#________________________________ LSTM

name_model = "s2s_LSTM_256TimeD_6"

keras.backend.clear_session()

# Definir hiperparâmetros
learning_rate = 0.0001
l2_reg = 0.00001
dropout_rate = 0.001
func_act='sigmoid'

batch_size = 128
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 3
sampling_rate=1



# Definir a entrada da rede
inputs = keras.layers.Input(shape=[None, num_inputs])

# LSTM
reshaped_inputs = keras.layers.Reshape((-1, num_inputs))(inputs)
lstm = keras.layers.LSTM(375, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(reshaped_inputs)
#lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
#lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.LSTM(160, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)
#lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.LSTM(256, return_sequences=True, kernel_regularizer=keras.regularizers.L2(l2_reg))(lstm)
#lstm = keras.layers.Dropout(rate=dropout_rate)(lstm)

# TimeDistributed Dense
timed = keras.layers.TimeDistributed(keras.layers.Dense(75))(lstm)
timed = keras.layers.TimeDistributed(keras.layers.Dropout(rate=dropout_rate))(timed)


# LSTM
lstm = keras.layers.LSTM(15, return_sequences=False, kernel_regularizer=keras.regularizers.L2(l2_reg))(timed)
output_lstm = keras.layers.Dense(num_targets)(lstm)


# Definir o modelo com a entrada e saída
model = keras.models.Model(inputs=inputs, outputs=output_lstm)

# Definir o otimizador com taxa de aprendizado definida
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


# Compilar o modelo com a função de perda, otimizador e métrica desejados
model.compile(loss=keras.losses.logcosh, optimizer=optimizer, metrics=["mse"])

# Adicionar callbacks para o modelo
model_checkpoint = keras.callbacks.ModelCheckpoint(name_model, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=32)

# Treinar o modelo
model.fit(train_set,
          steps_per_epoch= len(train)//batch_size,
          epochs=500,
          validation_data=valid_set,
          validation_steps=len(valid)//batch_size,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/500
781/781 [==============================] - 23s 25ms/step - loss: 0.0014 - mse: 0.0015 - val_loss: 0.0035 - val_mse: 0.0057


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_6\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_6\assets


Epoch 2/500
781/781 [==============================] - 19s 24ms/step - loss: 0.0011 - mse: 8.9844e-04 - val_loss: 0.0014 - val_mse: 0.0016


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_6\assets


...

INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_6\assets


Epoch 148/500
781/781 [==============================] - 22s 28ms/step - loss: 2.8162e-05 - mse: 3.4521e-05 - val_loss: 2.3764e-05 - val_mse: 2.5846e-05
Epoch 149/500
781/781 [==============================] - 24s 31ms/step - loss: 3.2242e-05 - mse: 4.2909e-05 - val_loss: 2.5998e-05 - val_mse: 3.0510e-05
Epoch 150/500
781/781 [==============================] - 20s 26ms/step - loss: 2.9519e-05 - mse: 3.7616e-05 - val_loss: 2.3281e-05 - val_mse: 2.5209e-05
Epoch 151/500
781/781 [==============================] - 21s 26ms/step - loss: 2.7195e-05 - mse: 3.3138e-05 - val_loss: 3.0563e-05 - val_mse: 4.0025e-05
Epoch 152/500
781/781 [==============================] - 20s 26ms/step - loss: 3.1456e-05 - mse: 4.1884e-05 - val_loss: 2.6904e-05 - val_mse: 3.2870e-05
Epoch 153/500
781/781 [==============================] - 21s 26ms/step - loss: 2.9224e-05 - mse: 3.7567e-05 - val_loss: 2.0663e-05 - val_mse: 2.0495e-05


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_6\assets


INFO:tensorflow:Assets written to: s2s_LSTM_256TimeD_6\assets


Epoch 154/500
781/781 [==============================] - 20s 25ms/step - loss: 2.7335e-05 - mse: 3.3957e-05 - val_loss: 2.7093e-05 - val_mse: 3.3524e-05
Epoch 155/500
781/781 [==============================] - 20s 25ms/step - loss: 3.0369e-05 - mse: 4.0245e-05 - val_loss: 3.7474e-05 - val_mse: 5.4445e-05
Epoch 156/500
781/781 [==============================] - 20s 26ms/step - loss: 2.8664e-05 - mse: 3.6990e-05 - val_loss: 5.2228e-05 - val_mse: 8.4065e-05
Epoch 157/500
781/781 [==============================] - 19s 25ms/step - loss: 2.7668e-05 - mse: 3.5174e-05 - val_loss: 1.0221e-04 - val_mse: 1.8418e-04
Epoch 158/500
781/781 [==============================] - 20s 25ms/step - loss: 2.9579e-05 - mse: 3.9192e-05 - val_loss: 1.1449e-04 - val_mse: 2.0890e-04
Epoch 159/500
781/781 [==============================] - 19s 25ms/step - loss: 2.8086e-05 - mse: 3.6371e-05 - val_loss: 9.5962e-05 - val_mse: 1.7202e-04
Epoch 160/500
781/781 [==============================] - 19s 25ms/step - loss: 2.7

In [56]:
name_model

's2s_LSTM_256TimeD_6'

In [57]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 3s 15ms/step - loss: 6.0235e-05 - mse: 1.0420e-04


In [58]:
model = keras.models.load_model(name_model)

In [59]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 12s 16ms/step - loss: 3.0426e-05 - mse: 4.0030e-05


### Avaliação

In [44]:
name_model

's2s_LSTM_320'

In [45]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 5s 29ms/step - loss: 0.0029 - mse: 0.0057


In [46]:
model = keras.models.load_model(name_model)

In [47]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 7s 30ms/step - loss: 1.3018e-04 - mse: 8.4895e-05


In [39]:
name_model

's2s_LSTM_2'

In [40]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 3s 15ms/step - loss: 1.2440e-04 - mse: 1.1023e-04


In [41]:
model = keras.models.load_model(name_model)

In [42]:
loss, metric = model.evaluate(test_set, steps=len(test)//batch_size)

160/160 [==============================] - 4s 15ms/step - loss: 2.4870e-04 - mse: 3.4340e-04


In [ ]:
name_model 

In [ ]:
model = keras.models.load_model(name_model)

### Predições

In [ ]:
predictions = model.predict(test_set, steps=len(test)//batch_size)

In [ ]:
predictions2 = model.predict(test_set, steps=len(test)//batch_size*2)

In [ ]:
predictions3 = model.predict(test_set, steps=1)

In [ ]:
predictions4 = model.predict(test_set_2, steps=1)

In [ ]:
predictions4

In [ ]:
test_2 = test[17276:]

In [ ]:
batch_size = 1
window_size = 32
num_inputs  = train.shape[1]
num_targets = 3
stride = 1
sampling_rate=1

In [ ]:
test_set_2 = sliding_window_sequence(test_2, window_size=window_size, num_inputs=num_inputs, num_targets=num_targets, stride=stride, batch_size=batch_size, sampling_rate=sampling_rate)

In [ ]:
len(test_2)

In [ ]:
predictions3

In [ ]:
predictions.shape

In [ ]:
predictions

In [ ]:
steps=len(test)//batch_size

In [ ]:
x = 1%1

In [ ]:
x

In [ ]:
steps

In [ ]:
len(test)

In [ ]:
len(predictions2)

In [ ]:
len(predictions)

In [ ]:
train2 = winn.dropna()

In [ ]:
train2

In [ ]:
train2 = train2.drop(['high', 'low','close', 'hl'], axis=1)

In [ ]:
mvhl = scaler.fit_transform(train2)

In [ ]:
train

In [ ]:
test

In [ ]:
scaler_target.fit_transform(data[:, -1].reshape(-1, 1))

In [ ]:
predict= scaler.inverse_transform(predictions4)

In [ ]:
winj

In [ ]:
test

In [ ]:
test_2

In [ ]:
predict